In [69]:
import pandas as pd
from tqdm import tqdm

from core.embedding_models import FastTextEmbeddingModel, EmbeddingModel
from core.models import LinearRegressionModel, CatboostRegressionModel

tqdm.pandas()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
df = pd.read_csv("../data/data_vacancies.csv")
df.head()

,id,custom_position,schedule,salary_from,salary_to,salary_pay_type,offer_education_id,education_name,education_is_base,education_order_num,city_id,list_regions,work_skills,tags_id
0,48202096,Сварщик-сборщик,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['сварочные работы', 'сборка изделий по чертеж...",NaN
1,48202097,Сварщик-монтажник,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['монтажные работы', 'строительные работы', 'э...",NaN
2,48202098,Слесарь-сборщик,полный рабочий день,60000,80000,NaN,0,любое,True,0,2,[4],"['работа на фрезерных станках', 'слесарный рем...",NaN
3,48202356,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,1,[3],"['комплектация товара', 'маркировка', 'стрессо...","[6, 9]"
4,48202357,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,57,"[181, 182, 183, 185, 186, 187, 188, 189, 190, ...","['маркировка', 'стрессоустойчивость', 'погрузо...","[6, 9]"


In [63]:
df["salary_mean"] = (df.salary_from + df.salary_to) / 2
df["target"] = df.salary_mean
df = df[df.salary_mean < df.salary_mean.quantile(0.95)]

cols_to_concat = ["custom_position"]
df["combined_string"] = df[cols_to_concat].agg("\n".join, axis=1)


In [75]:
dataset = df.iloc[:]
# embedding_model = FastTextEmbeddingModel()
embedding_model = EmbeddingModel()

In [76]:
lr_model = LinearRegressionModel(embedding_model=embedding_model)


In [77]:
dataset.loc[:, "emb"] = dataset["combined_string"].progress_apply(lambda x: embedding_model.generate(x))


100%|██████████| 18468/18468 [00:49<00:00, 372.64it/s]


In [78]:
dataset["emb"].iloc[0].shape

(312,)

In [79]:
lr_model.train(dataset)

Model trained
Test score is 15864.476296020573


In [80]:
cb_model = CatboostRegressionModel(embedding_model=embedding_model)


In [81]:
cb_model.train(dataset, metric_period=100)

Learning rate set to 0.062656
0:	learn: 26518.8973307	total: 16.7ms	remaining: 16.7s
100:	learn: 19620.0188419	total: 1.22s	remaining: 10.9s
200:	learn: 18049.7677235	total: 2.36s	remaining: 9.39s
300:	learn: 16879.7865335	total: 3.48s	remaining: 8.09s
400:	learn: 16094.6586532	total: 4.64s	remaining: 6.93s
500:	learn: 15515.9058553	total: 5.76s	remaining: 5.74s
600:	learn: 15067.7585683	total: 6.91s	remaining: 4.59s
700:	learn: 14706.9670582	total: 8.05s	remaining: 3.43s
800:	learn: 14420.7873090	total: 9.19s	remaining: 2.28s
900:	learn: 14178.1286967	total: 10.3s	remaining: 1.14s
999:	learn: 13972.9771133	total: 11.4s	remaining: 0us
Model trained
Test score is 12521.844368893324
